# Health data preprocessing and input modelling exercises.

**In this lab you will:**

* Gain practical knowledge in pre-processing and analysing real world stochastic health system data
* Learn how to fit distribution to data
* Learn how to select a suitable distribution for your data

> **STUDENT BEWARE**: This lab can be very frustrating! It is designed to show you the sort of data wrangling, analysis and modelling decisions/assumptions you may need to perform in real simulation study.  But do persevere with it (answers are available as well!).  The experience should demonstrate that fitting distributions to real data is difficult and not quite as textbooks make out! By the end of the lab both your `pandas` skills and simulation input modelling skills will have improved.  >_<


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#auto_fit function from pythonhealthcare.org
from input_modelling.fitting import auto_fit

sns.set()

## **Exercise 1**: stroke ward admissions and early supported discharge

The dataset we will work with represents admissions and discharges at a hospital acute stroke ward.  

The dataset contains the following fields

* **patient_type**: stroke, tia (minor stroke), complex-neurological (e.g. brain injury) and other (i.e. non-neurological patients) (str)
* **hosp_arrival_date**: date dd-mmm-yy of arrival to hospital (str)
* **hosp_arrival_time**: time HH:MM of arrival (str)
* **asu_admit_date**: date dd-mmm-yy of admission to the acute stroke unit (str)
* **asu_discharge_date**: date dd-mmm-yy of discharge from the acute stroke unit (str)
* **wardN_admit_date**: date dd-mmm-yy of admission to the ward N where N is between 2 and 4 (str)
* **wardN_discharge_date**: date dd-mmm-yy of discharge from the ward N where N is between 2 and 4 (str)
* **hosp_discharge_date**: date dd-mmm-yy of discharge from hospital (str)
* **hosp_discharge_time**: time HH:MM of discharge from hospital (str)
* **esd**: Early supported discharge binary 0/1. 1 == patient underwent ESD

**In this exercise you will need to:**
* Make some assumptions about how to model LoS.
* Preprocess and wrangle the data set
* Perform some exploratory analysis
* Fit one or more distributions to decide how to model the arrival process
* Fit one or more distributions to decide how to model length stay on the ward.

### Reading in the data

In [ ]:
acute_strokes = pd.read_csv('../../../../input_modelling_data/input_data_asu.csv')

In [ ]:
acute_strokes.shape

In [ ]:
#take a look at the head of the dataset
acute_strokes.head()

In [ ]:
#any missing data?
acute_strokes.info()

# PART 1: PRE-PROCESSING THE DATA

## **Exercise 1.1**: reformat the date fields

The date fields as-is are not suitable for analysis in python. Before we can do any analysis we need to pre-process the dates.

**Task**:

* Convert `acute_admit` and `asu_discharge` into valid date fields: format = YYYY-MM-DD

**Hints**

* Take a look at the `pd.to_datetime()` function
* The dates are in UK format.  You will need to look at what options `pandas` provides to handle it.

In [ ]:
# your code here ...

In [ ]:
#example answer

date_fields = ['hosp_arrival_date', 
               'asu_admit_date',
               'asu_discharge_date',
               'hosp_discharge_date']

for field in date_fields:
    acute_strokes[field] = pd.to_datetime(acute_strokes[field], dayfirst=True)


In [ ]:
acute_strokes.head()

In [ ]:
acute_strokes['hosp_arrival_dt'] = pd.to_datetime(acute_strokes['hosp_arrival_date'].astype('str') + ' ' + acute_strokes['hosp_arrival_time'])
acute_strokes.head()

In [ ]:
acute_strokes['hosp_discharge_dt'] = pd.to_datetime(acute_strokes['hosp_discharge_date'].astype('str') + ' ' + acute_strokes['hosp_discharge_time'])
acute_strokes.head()

In [ ]:
acute_strokes.info()

## **Exercise 1.2** Sort the admissions by date

**Task**:
* Put the data into ascending order sorted by`asu_admit` 

**Hints**:
* Take a look at `pd.DataFrame.sort_values(by)`
* Make sure you take a look at what `sort_values` does and what it returns

In [ ]:
#your code here ...

In [ ]:
#example answer
acute_strokes = acute_strokes.sort_values(by='hosp_arrival_dt')
acute_strokes.head()

## **Exercise 1.3** Recoding the `patient_type` field

It is not strictly necessary, but its often useful to get recode fields of type **str** to an **int**

Let's take a look at the unique codings we have for patient_type:

In [ ]:
acute_strokes.patient_type.unique()

So we have four types of basic classification.  Let's recode them as follows:
* Stroke: 0
* Complex-neurological: 1
* Other: 2
* TIA: 3

**Task:**

* Either recode or create a new field that encodes the type of patient as above. The output of your preprocessing will be a numeric field.
* Confirm your recoding has worked (you can always reload the data an preprocess again if needed!)

**Hints:**

* There are multiple ways to complete this exercise.  One option is to make use of `pd.DataFrame.replace()` method.  For that you will need to create a dict that maps a str to an int e.g.

```python
example_mapping = {'foo':0,
                   'bar':1,
                   'spam':2}

example_df.replace(example_mapping)
```

In [ ]:
# your code here ...

In [ ]:
#example answer ..
#we will store the recoding in a 
recoded_patient_type = {'Stroke':0, 
                        'Complex-neurological':1, 
                        'TIA':2, 
                        'Other':3}

In [ ]:
acute_strokes = acute_strokes.replace(recoded_patient_type)

In [ ]:
acute_strokes.head()

## Exercise 1.4: Drop the redundant fields 

**Task:**
* Drop the redundant fields (provided in a list below) from the dataframe.
* Save the to cleaned data a new CSV file `clean_arrival_data_student.csv` 

In [ ]:
to_drop = ['hosp_arrival_date', 'hosp_arrival_time',
       'asu_admit_date', 'asu_discharge_date', 'ward2_admit_date',
       'ward2_discharge_date', 'ward3_admit_date', 'ward3_discharge_date',
       'ward4_admit_date', 'ward4_discharge_date', 'hosp_discharge_date',
       'hosp_discharge_time']

#your code here

In [ ]:
to_drop = ['hosp_arrival_date', 'hosp_arrival_time',
       'asu_admit_date', 'asu_discharge_date', 'ward2_admit_date',
       'ward2_discharge_date', 'ward3_admit_date', 'ward3_discharge_date',
       'ward4_admit_date', 'ward4_discharge_date', 'hosp_discharge_date',
       'hosp_discharge_time']

acute_strokes = acute_strokes.drop(to_drop, axis=1)

In [ ]:
acute_strokes.head()

In [ ]:
#acute_strokes.to_csv('cleaned_arrival_data_student.csv', index=False)

---
# PART 2: INTER-ARRIVAL TIMES

## **Exercise 2.1** Explore the interarrival distributions

A first task in input modelling is often to explore the interarrival distributions.  This is often a major source of variability in stochastic health systems.

**Task:**

* Using your **pre-sorted** data calculate the days between arrivals.  We will ignore patient types for the moment.
* Plot a histogram of interarrival times
* Calculate the mean and stdev of the interarrival times
* What might be a reasonable distribution?
* What does the histogram suggest might be going on in the data?

**Hints**:

* Checkout the `pd.DataFrame.diff()` method
* This will return `timedelta64`.  To convert to hours you need to divide by:

```python
np.timedelta64(24, 'h')
```

In [ ]:
#code to load cleaned dataset.
acute_strokes = pd.read_csv('precleaned_arrival_data.csv', 
                             parse_dates=['hosp_arrival_dt', 
                                          'asu_admit_dt', 
                                          'hosp_discharge_dt'])
acute_strokes.info()

In [ ]:
#your code here ...

In [ ]:
#example answer ...
inter_arrivals = acute_strokes['hosp_arrival_dt'].diff() / np.timedelta64(24, 'h')
inter_arrivals.head()

In [ ]:
inter_arrivals.hist(bins=100)

In [ ]:
inter_arrivals.mean()

In [ ]:
inter_arrivals.std()

## **Exercise 2.2** Subgroup interarrival distributions.

We know that there are at least four categories of patients that use the system.  The histogram of the overall population suggests that we should explore if the distributions vary by group.

**Task:**
* For each sub group (stroke, complex, tia and other)
    * summarise the subgroup sample size
    * calculate the subgroups interarrival days (i.e. the days between consecutive stroke arrivals)
    * plot the distribution as a histogram
    * calculate the mean and standard dev
    
* Without doing a formal statistical test, what distributions do you think would be useful?

**Hints:**

* You can plot each of histograms seperately.  But if you wanted to plot histograms in a grid remember `Matplotlib` has `fig, ax = plot.subplots(nrows, ncols)` where `ax` is a numpy matrix and each element represents the corresponding axis of a chart in the grid.
* When plotting histograms you could try different numbers of bins.  A good starting number is 20.

In [ ]:
# your code here ...

In [ ]:
#example answer ...
iat_by_group = {}

for patient_type in range(4):
    #calculate individual iat
    data = acute_strokes.loc[acute_strokes['patient_type'] \
            == patient_type]['hosp_arrival_dt'].diff() / np.timedelta64(24, 'h')

    #print mean
    print(f'{patient_type}: n:{data.shape[0]}, m:{data.dropna().mean():.2f}, s:{data.dropna().std():.2f}')
    
    #store by group
    iat_by_group[f'{patient_type}'] = data


In [ ]:
#plot histograms on grid sharing the same x-axis
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 5), sharex=True)

key = 0
for row in range(2):
    for col in range(2):
        #plot histogram dropping initial NAN to avoid matplotlib warning/moaning
        ax[row][col].hist(iat_by_group[f'{key}'].dropna(), bins=20, 
                          density=True)
        key += 1

In [ ]:
#if you wanted to store subgroup means
means = [items.dropna().to_numpy().mean() for group, items in iat_by_group.items()]
means

In [ ]:
#and subgroup std
stds = [items.dropna().to_numpy().std() for group, items in iat_by_group.items()]
stds

## **Exercise 2.3** Interarrivals: Stroke with ESD versus Stroke with no-ESD

Let's explore if Early Supported Discharge patients have a different arrival process to non-ESD patients

**Task:**

* Using the field 'esd' split the **stroke** patients into two groups:  ESD stroke patients and non-ESD patients.
* Plot the distributions
* Calculate the mean and standard deviation of each subgroup.
* You should find that the mean and stdev are roughly the same for each group.  
    * What distribution does this suggest is a good choice and what is special about this distribution that means you don't need to model it as two seperate arrival processes?
    * What percentage of stroke arrivals go on to have ESD?  How might you model that?
* Using `auto_fit()` test if the exponential distribution is rejected

In [ ]:
# your code here ...

In [ ]:
inter_arrivals_esd = acute_strokes.loc[(acute_strokes['esd'] == 1) & (acute_strokes['patient_type'] == 0)]['hosp_arrival_dt'].diff() / np.timedelta64(1, 'h')
inter_arrivals_esd.shape

In [ ]:
inter_arrivals_no_esd = acute_strokes.loc[(acute_strokes['esd'] == 0) & (acute_strokes['patient_type'] == 0)]['hosp_arrival_dt'].diff() / np.timedelta64(1, 'h')
inter_arrivals_no_esd.shape

In [ ]:
#prop of arrivals that are ESD
inter_arrivals_esd.shape[0] / (inter_arrivals_esd.shape[0] + inter_arrivals_no_esd.shape[0])

In [ ]:
#plot histograms on grid
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 5), sharey=True, sharex=True)

ax[0].hist(inter_arrivals_no_esd.dropna(), density=True);
ax[1].hist(inter_arrivals_esd.dropna(), density=True);

In [ ]:
inter_arrivals_esd.describe()

In [ ]:
inter_arrivals_no_esd.describe()

In [ ]:
#doesn't reject exponential/
auto_fit(inter_arrivals_no_esd, hist=True, pp=True, dist_names=['expon'])

**possible answer**

The mean and the std of the ESD and non-ESD arrivals are for practical purposes the equal. Auto-fit suggests does not rule out an exponential distribution (i.e. a poission process).  Therefore an option would be to model both of these populations as a single arrival process where on arrival patients are labelled either 1 for ESD or for 0 ESD following a Bernoulli distribution. 

___

# PART 3: LENGTH OF STAY

To estimate distributions of length of stay we need to calculate the difference in days between admission and discharge.

**Before we do this there are a few complications in the dataset that we will need to manage as best we can**. The wrangled data is provided in `asu_discharges.csv`

Issues with current data:
* The data only contain a date stamp for admission to the ASU (there is no time), but we do have a hospital arrival timestamp. These can be on the same day or several (or many) days apart.
* The data include a hospital discharge datetime stamp BUT this does not always represent a discharge from ASU.
* This is because some of the patients in the data are transfered from the ASU to multiple hospital wards before they are discharged.  
    
>To handle these issues we will make some assumptions.  We'll assume that the sample of patients that only stayed on the asu is representative of the whole population and also make use of a target for getting admitting patients onto a stroke unit within 4 hours of arrival.

This is what we will do:

* We will limit our analysis of LoS to patients who only stay on a **single** ward. 
* If patients are admitted to the ASU on a different day from arrival to hospital we will assume admission time is midnight.
* If patients are admitted to the ASU on the same day as arrival to hospital we will assume patients are admitted within 4 hours of their hospital arrival unless that crosses midnight and then we will set it to 23:59.

**Note: In practice we would analyse these implications of these decisions during modelling through a process called *sensitivity analysis*.**

## **Exercise 3.1**: LoS by `patient_type`

**Task:**

* Load `asu_discharges.csv` (code provided below)
* Create a new field called `los` that is equal the patients length of stay in the asu **in days.**
* Explore the distribution of LoS by `patient_type` subgroup.

**Hints:**

* In `pandas` when you calculate the difference between two `datetime64` values you are returned a `timedelta64`.  To convert this into days you need to divide by `np.timedelta64(24, 'h')`

In [ ]:
#load data
asu_discharges = pd.read_csv('../../../../input_modelling_data/asu_discharges.csv', 
                             parse_dates=['hosp_arrival_dt', 
                                          'asu_admit_dt', 
                                          'hosp_discharge_dt'])
asu_discharges.info()

In [ ]:
#your code here to calculate the difference in days

In [ ]:
#example answer
asu_discharges['los'] = (asu_discharges['hosp_discharge_dt'] - \
                    asu_discharges['asu_admit_dt']) / np.timedelta64(24, 'h')

asu_discharges.head()

In [ ]:
#loop through and create LoS by subgroup
los_by_group = {}

for patient_type in range(4):
    #calculate individual los
    data = asu_discharges.loc[asu_discharges['patient_type'] \
            == patient_type]['los']

    #print mean
    print(f'{patient_type}: n:{data.shape[0]}, m:{data.dropna().mean():.2f}, s:{data.dropna().std():.2f}')
    
    #store by group
    los_by_group[f'{patient_type}'] = data

In [ ]:
#plot histograms on grid
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 5), sharex=True)

key = 0
for row in range(2):
    for col in range(2):
        #plot histogram dropping initial NAN to avoid matplotlib warning/moaning
        ax[row][col].hist(los_by_group[f'{key}'].dropna(), bins=50)
        key += 1

In [ ]:
#auto fit

key = 0
for key in range(4):
    auto_fit(los_by_group[f'{key}'].dropna(), hist=False)
        

**Conclusions:** There is no perfect distribution here.  But lognormal seems to be the best fitting.


## Exercise 3.2. LoS ESD versus Non-ESD

**Task**:
* Using the `esd` and `patient_type` fields split the dataset into strokes with ESD and strokes without ESD.  
* Generate summary statistics for each group.
* Use auto-fit 
* Which distributions do you think are a good choice?

In [ ]:
# your code here ...

In [ ]:

#Los distributions for esd
los_stroke_esd = asu_discharges.loc[(asu_discharges['esd'] == 1) \
                                & (asu_discharges['patient_type'] == 0)]['los']

los_stroke_no_esd = asu_discharges.loc[(asu_discharges['esd'] == 0) \
                                & (asu_discharges['patient_type'] == 0)]['los']

print('ESD LoS summary')
print(los_stroke_esd.describe())
print('\n------------------\nNO ESD LOS summary')
print(los_stroke_no_esd.describe())

In [ ]:
#plot histograms on grid
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 5), sharey=True, sharex=True)

ax[0].hist(los_stroke_esd.dropna(), density=True, bins=40);
ax[1].hist(los_stroke_no_esd.dropna(), density=True, bins=40);

In [ ]:
dist_names = ['beta',
              'expon',
              'lognorm',
              'norm',
              'pearson3',
              'weibull_min', 
              'weibull_max']

auto_fit(los_stroke_no_esd, hist=True, pp=False, dist_names=dist_names)

In [ ]:
auto_fit(los_stroke_esd, hist=True, pp=False, dist_names=dist_names)

### Conclusions. ESD versus no ESD los.  

No distribution is a perfect fit due to high kertosis, but lognormal is reasonable.  

## END OF LAB